## 우루사 VS 고덱스 분석하기
http://medinavi.co.kr/search_medicine.asp?keyword1=&keyword2=%EC%9A%B0%EB%A3%A8%EC%82%AC&keyword3=%EB%8C%80%EC%9B%85%EC%A0%9C%EC%95%BD

```
우루사정100밀리그램 246501ATB
우루사정200밀리그램 246502ATB
우루사정300밀리그램 246506ATB
우루사100mg연질캅셀 246501ACS
우루사캅셀200mg 246502ACH
우루사캡슐250밀리그램 246503ACH
고덱스캡슐 427800ACH
```


<img src="https://i.imgur.com/4dEl5Dl.jpg">

출처 : http://www.docdocdoc.co.kr/news/articleView.html?idxno=1053667

## 라이브러리로드

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

## 시각화를 위한 한글폰트 설정

In [ ]:
import matplotlib.pyplot as plt

# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic') 
plt.rc('axes', unicode_minus=False)

In [ ]:
from IPython.display import set_matplotlib_formats
# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
set_matplotlib_formats('retina')

## 데이터셋 로드

In [ ]:
df = pd.read_csv('data/medicine/medicine_2011_2018.csv', encoding="cp949")
df.shape

## 데이터 미리보기

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

## 데이터 전처리

### 주성분코드로 약품명 찾기
* 우루사정100밀리그램 246501ATB
* 우루사정200밀리그램 246502ATB
* 우루사정300밀리그램 246506ATB
* 우루사100mg연질캅셀 246501ACS
* 우루사캅셀200mg 246502ACH
* 우루사캡슐250밀리그램 246503ACH
* 고덱스캡슐 427800ACH

In [ ]:
code = """우루사정100밀리그램 246501ATB
우루사정200밀리그램 246502ATB
우루사정300밀리그램 246506ATB
우루사100mg연질캅셀 246501ACS
우루사캅셀200mg 246502ACH
우루사캡슐250밀리그램 246503ACH
고덱스캡슐 427800ACH"""
code

In [ ]:
# 위에서 만든 텍스트로 주성분코드와 약품명을 매치하기 위해 딕셔너리를 만듭니다.
# code_key_val 이라는 변수에 딕셔너리 형태로 값을 넣어줍니다.
codes = code.split("\n")
code_key_val = {}
for cname in codes:
    cname = cname.split(" ")
    code_key_val[cname[-1]] = cname[0]

print(code_key_val)

In [ ]:
# 딕셔너리를 매핑합니다.
df["약품일반성분명"] = df["약품일반성분명코드"].map(code_key_val)

df[["약품일반성분명코드", "약품일반성분명"]].head()

### 우루사와 고덱스 구분하기

In [ ]:
# 우루사는 "약품일반성분명코드"가 여러 개 입니다.
# 고덱스코드를 통해 우루사인지 아닌지 True, False로 boolean 값을 갖는 컬럼을 생성합니다.
# 컬럼명은 "우루사"로 합니다.
df["우루사"] = df["약품일반성분명코드"] != "427800ACH"
df[["약품일반성분명코드", "약품일반성분명", "우루사"]].head()

### 결측치 보기

In [ ]:
# 결측치가 있는지 봅니다.
df.isnull().sum()

### 요양개시일자로 정렬하기

In [ ]:
# "요양개시일자", "가입자일련번호", "처방내역일련번호", "일련번호" 로 정렬합니다.
df = df.sort_values(
    ["요양개시일자", "가입자일련번호", "처방내역일련번호", "일련번호"])
df.head()

### 날짜 다루기
* 연, 월, 일, 요일 추출하기

In [ ]:
# "요양개시일자"의 데이터 타입을 봅니다.
df["요양개시일자"].head(1)

In [ ]:
# "요양개시일자"를 datetime 형태로 변환하기 위해서 astype 을 사용해 문자형태로 변환합니다.
# 타입을 변경하고 다시 같은 컬럼에 데이터를 담아주어야 변경된 내용이 반영됩니다.
df["요양개시일자"] = pd.to_datetime(df["요양개시일자"].astype(str))

In [ ]:
# "요양개시일자"에서 연, 월, 일을 추출합니다.
df["연"] = df["요양개시일자"].dt.year
df["월"] = df["요양개시일자"].dt.month
df["일"] = df["요양개시일자"].dt.day

In [ ]:
# "요양개시일자"에서 "요일"과 "영문요일"을 추출합니다.
df["요일"] = df["요양개시일자"].dt.dayofweek
df["영문요일"] = df["요양개시일자"].dt.day_name()

In [ ]:
# 요일을 한글요일로 변경합니다.
weekday = "월화수목금토일"
df["요일명"] = df["요일"].map(lambda x : weekday[x])

df[["요일", "영문요일", "요일명"]].head()

### 컬럼 디스플레이 옵션 변경하기

In [ ]:
# 컬럼이 많이 늘어나서 한번에 모든 컬럼이 보이지 않습니다.
# max_columns 의 수를 지정합니다.
pd.options.display.max_columns = 30

In [ ]:
# sample 을 통해 모든 컬럼이 보이는지 확인합니다.
df.sample()

### 시도코드를 시도명으로 변환하기
* 시도코드를 통해 시도명을 만듭니다.

In [ ]:
city = """11 서울특별시
42 강원도
26 부산광역시
43 충청북도
27 대구광역시
44 충청남도
28 인천광역시
45 전라북도
29 광주광역시
46 전라남도
30 대전광역시
47 경상북도
31 울산광역시
48 경상남도
36 세종특별자치시
49 제주특별자치도
41 경기도"""

In [ ]:
city_list = city.split("\n")
city_name = {}
for city_info in city_list:
    city_split = city_info.split()
    city_key = int(city_split[0])
    city_val = city_split[-1]
    city_name[city_key] = city_val
    
print(city_name)

In [ ]:
df["시도명"] = df["시도코드"].map(city_name)
df[["시도명", "시도코드"]].head()

### '연령대코드(5세단위)' 를 연령대로 변환하기

In [ ]:
# '연령대코드(5세단위)' 를 연령대로 나타내기
age_code = """1 00~04세 
2 05~09세
3 10~14세
4 15~19세
5 20~24세
6 25~29세
7 30~34세
8 35~39세
9 40~44세
11 50~54세
10 45~49세
12 55~59세
13 60~64세
14 65~69세
15 70~74세
16 75~79세
17 80~84세
18 85세+"""

In [ ]:
age_list = age_code.split("\n")
age_dict = {}
for age_pair in age_list:
    age_split = age_pair.split(" ")
    key = age_split[0]
    key = int(key)
    val = age_split[1]
    age_dict[key] = val

In [ ]:
age_dict

In [ ]:
df["연령대"] = df["연령대코드(5세단위)"].map(age_dict)

### 특정 약품일반성분명코드로 서브셋 만들기
* isin 사용하기

In [ ]:
# 여러 개를 찾을 때
df_sample = df[df["약품일반성분명코드"].isin(["246506ATB", "246503ACH"])]
df_sample.shape

## 파일로 저장하기

In [ ]:
# 파일로 저장하기
df_sample.to_csv("sample.csv", index=False, encoding="cp949")

In [ ]:
# 저장된 파일 읽어와서 확인하기
pd.read_csv("sample.csv", encoding="cp949").head()

## 연도별 처방 수

In [ ]:
# 범주형 vs 범주형 값을 구할 때 crosstab 을 사용합니다.
# 연도별로 우루사와 고덱스의 처방수를 구합니다.
pd.crosstab(df["연"], df["우루사"])

In [ ]:
# 위에서 구한 값을 pivot_table 로 똑같이 구해봅니다.
# 연도별로 우루사와 고덱스의 처방수를 구합니다.
# aggfunc 에 len 을 사용하면 갯수를 카운트 합니다.
# "count"를 사용해도 됩니다.
year_ursa = pd.pivot_table(data=df, index=["연", "우루사"],  
               values="가입자일련번호", aggfunc="count")
year_ursa.columns = ["처방수"]
year_ursa = year_ursa.reset_index()
year_ursa

In [ ]:
# 위에서 구한 값을 pointplot으로 시각화 합니다.
sns.pointplot(data=year_ursa, x="연", y="처방수", hue="우루사")

In [ ]:
# 위에서 구한 값을 countplot으로 시각화 합니다.
sns.countplot(data=df, x="연", hue="우루사")

## 연도별 처방 금액

In [ ]:
# 연도별 처방금액의 합계를 구합니다.
year_count_sum = pd.pivot_table(data=df, index="연", values="금액", 
                          aggfunc="sum")
year_count_sum["금액"].map(lambda x: f"{x:,.0f}")

In [ ]:
# 위에서 구한 값을 시각화 합니다.
year_count_sum.plot()

In [ ]:
# 우루사 vs 고덱스의 연도별 처방 금액을 구합니다.

year_ursa = pd.pivot_table(data=df, index=["연", "우루사"],  
               values="가입자일련번호", aggfunc="sum")
year_ursa.columns = ["처방금액"]
year_ursa = year_ursa.reset_index()
year_ursa

In [ ]:
# 위에서 구한 값을 pointplot으로 시각화 합니다.
sns.pointplot(data=year_ursa, x="연", y="처방금액", hue="우루사")

### 시도별 처방수와 금액 구하기

In [ ]:
year_ursa = pd.pivot_table(data=df, index="연",
                           columns = "시도명",
                           values="가입자일련번호", aggfunc="count")
year_ursa.fillna(0).astype(int).style.bar()

In [ ]:
year_ursa.fillna(0).astype(int).style.background_gradient()

### 시도별 서브플롯 그리기

In [ ]:
# catplot을 통해 연도별, 금액을 시도명으로 서브플롯을 그립니다.
# 이 때, 우루사 여부에 따라 다른색상으로 pointplot을 그립니다.
sns.catplot(data=df, x="연", y="금액", 
            hue="우루사", col="시도명", col_wrap=4, 
            kind="point", ci=None)

## 연도, 월별 처방수와 금액

In [ ]:
group = df.groupby(["월", "연"])["금액"].agg(["count", "sum"]).unstack()
group["sum"].style.format("{:,}")

* 스타일 적용 : https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

In [ ]:
# background_gradient 스타일을 적용해서 값의 많고 적음에 따라 색상으로 구분되게 그립니다.
group["sum"].style.background_gradient(cmap="Blues")

In [ ]:
# 그룹화된 결과에서 처방수만 가져오기
# 수치의 높고 낮음을 색상으로 표현하기 위해 background_gradient 를 사용합니다.
group["count"].style.background_gradient(cmap="Blues")

In [ ]:
# 그룹화된 결과에서 처방금액 합계만 보기
# bar 를 통해 셀안에 그래프로 표현하기
group["sum"].style.bar()

In [ ]:
# 위에서 만든 변수에서  group["count"] 값만 가져와서 year_month_count 변수에 담습니다.
year_month_count = group["count"]
year_month_count

In [ ]:
# year_month_count 변수에 담긴 값을 heatmap 으로 그립니다.
# cmap 을 YlGn 으로 표시합니다.
plt.figure(figsize=(20, 4))
sns.heatmap(year_month_count, annot=True, cmap="YlGn", fmt=",.0f")

## 연, 월별 처방 수와 합계 금액
* 연, 월별 처방 수와 합계금액을 groupby 로 구해보세요.

In [ ]:
# 위에서 구한 값은 index와 columns에 연 또는 월이 들어갑니다.
# 같은 값을 갖는 데이터를 index 에 연, 월이 모두 들어가도록 groupby 로 연산을 해봅니다.
# index에 연, 월이 모두 들어갈 때 위의 형태의 데이터프레임과 다루는 차이를 알아봅니다.
# 금액 값에 대해 처방수와 금액의 합계를 구합니다.
df_group_year = df.groupby(["연", "월"])["금액"].agg(["count", "sum"])
df_group_year

In [ ]:
df_group_year["count"].plot()

In [ ]:
# 월별 처방금액 합계
plt.figure(figsize=(10, 4))
sns.pointplot(data=df, x="월", y="금액", hue="연", estimator=np.sum)

In [ ]:
# "약품일반성분명"에 따라 평균 처방 금액 보기
plt.figure(figsize=(10, 4))
sns.pointplot(data=df, x="월", y="금액", hue="약품일반성분명")

In [ ]:
# 월별 "약품일반성분명"에 따른 처방금액 합계
plt.figure(figsize=(10, 4))
sns.pointplot(data=df, x="월", y="금액", hue="약품일반성분명", estimator=np.sum)

In [ ]:
# 월별 "우루사" 여부에 따른 처방금액 합계
plt.figure(figsize=(10, 4))
sns.pointplot(data=df, x="월", y="금액", hue="우루사", estimator=np.sum)

In [ ]:
# 월별 처방수
plt.figure(figsize=(10, 4))
sns.pointplot(data=df, x="월", y="금액", hue="우루사", estimator=len)

## 요일별 처방수

In [ ]:
# countplot 을 통해 요일별 우루사 여부에 따른 처방수를 시각화 합니다.
# 이때, data에 df.sort_values("요일") 로 
# 요일별로 정렬된 데이터를 넣어주면 요일별로 정렬이 가능합니다.
sns.countplot(data=df.sort_values("요일"), x="요일명", hue="우루사")

In [ ]:
# "약품일반성분명코드" 별로 요일별 처방수를 시각화
sns.countplot(data=df.sort_values("요일"), x="요일명", hue="약품일반성분명코드")

### 연령대코드 vs 성별코드

In [ ]:
sns.countplot(data=df, x="연령대코드(5세단위)", hue="성별코드")

### 월 vs 성별코드

In [ ]:
sns.countplot(data=df, x="월", hue="성별코드")

## 우루사 vs 고덱스

### 성별코드별 처방수

In [ ]:
plt.figure(figsize=(15, 4))
sns.countplot(data=df, x="약품일반성분명", hue="성별코드")

In [ ]:
sns.countplot(data=df, x="우루사", hue="성별코드")

In [ ]:
sns.countplot(data=df, x="연", hue="우루사")

In [ ]:
plt.figure(figsize=(15, 4))
sns.countplot(data=df, x="월", hue="약품일반성분명")
plt.legend(bbox_to_anchor=(1, 1))

In [ ]:
sns.countplot(data=df, x="월", hue="우루사")

In [ ]:
plt.figure(figsize=(15, 4))
sns.countplot(data=df, x="일", hue="성별코드")

### 기간별 단가와 금액

In [ ]:
# 연도별 평균단가를 "성별코드"에 따라 다른 색상으로 그립니다.
sns.pointplot(data=df, x="연", y="단가", hue="성별코드", ci=None)

In [ ]:
# 연도별 평균금액을 "성별코드"에 따라 다른 색상으로 그립니다.
sns.pointplot(data=df, x="연", y="금액", hue="성별코드", ci=None)

In [ ]:
# 연도별 "1회투약량"을 "우루사"여부에 따라 다른 색상으로 그립니다.
sns.pointplot(data=df, x="연", y="1회투약량", hue="우루사", ci=None, 
              estimator=len)

In [ ]:
# 연도별 단가를 "우루사" 여부에 따라 다른 색상으로 그립니다.
sns.pointplot(data=df, x="연", y="단가", hue="우루사")

In [ ]:
# 월별 평균 처방 금액을 성별코드에 따라 다른 색상으로 그립니다.
sns.pointplot(data=df, x="월", y="금액", hue="성별코드")

In [ ]:
# 월별 평균 처방 금액을 우루사 여부에 따라 다른 색상으로 그립니다.
sns.pointplot(data=df, x="월", y="금액", hue="우루사")

### 성별코드별

In [ ]:
# 연도별 평균 총투여일수
sns.pointplot(data=df, x="연", y="총투여일수", hue="성별코드")

In [ ]:
# 연도별 평균 1일투약량
sns.pointplot(data=df, x="연", y="1일투약량", hue="성별코드")

In [ ]:
# 연도별 평균 연령대코드
sns.pointplot(data=df, x="연", y="연령대코드(5세단위)", hue="성별코드")

### 연령대코드(5세단위)

In [ ]:
sns.pointplot(data=df, x="연령대코드(5세단위)", y="총투여일수", hue="성별코드")

In [ ]:
sns.pointplot(data=df, x="연령대코드(5세단위)", y="금액", hue="성별코드")

In [ ]:
sns.pointplot(data=df, x="연령대코드(5세단위)", y="1일투약량", hue="성별코드")